# Notebook 1: Cleanlab for Approval Data Quality

**Objective**: Use Cleanlab to audit historical approval decisions and identify:
- Mislabeled approvals/rejections
- Inconsistent approver decisions
- Ambiguous cases that were decided inconsistently

This prepares clean training data for the IARS classifier.

---

## Flow Diagram

```mermaid
flowchart TD
    subgraph Input["📥 Input Data"]
        A[Historical Approval Data]
        B[Request Text + Decisions]
    end

    subgraph Embeddings["🔤 Text Processing"]
        C[Sentence Transformer]
        D[Text Embeddings]
    end

    subgraph ML["🤖 Machine Learning"]
        E[Train Classifier]
        F[Cross-Validated Predictions]
        G[Prediction Probabilities]
    end

    subgraph Cleanlab["🧹 Cleanlab Analysis"]
        H[get_label_quality_scores]
        I[find_label_issues]
        J[compute_confident_joint]
        K[health_summary]
    end

    subgraph Output["📤 Outputs"]
        L[Label Quality Scores]
        M[Issue Flags]
        N[Dataset Health]
        O[Cleaned Training Data]
    end

    subgraph YRSN["🎯 YRSN Signals"]
        P[label_quality → N component]
        Q[normalized_margin → S component]
        R[normalized_entropy → S component]
    end

    A --> C
    B --> C
    C --> D
    D --> E
    E --> F
    F --> G
    G --> H
    G --> I
    G --> J
    G --> K
    H --> L
    I --> M
    J --> N
    K --> N
    L --> O
    M --> O
    L --> P
    G --> Q
    G --> R

    style Cleanlab fill:#e1f5fe
    style YRSN fill:#fff3e0
    style Input fill:#e8f5e9
    style Output fill:#fce4ec
```

---

**Collapse Type Focus**: POISONING (mislabeled approval decisions)

**Difficulty**: ⭐ Easy

## 1. Setup

In [ ]:
# Install dependencies
!pip install cleanlab sentence-transformers scikit-learn pandas boto3 --quiet

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

from cleanlab.filter import find_label_issues
from cleanlab.rank import get_label_quality_scores
from cleanlab.count import compute_confident_joint
from cleanlab.dataset import health_summary

print("Dependencies loaded successfully")

## 2. Load Historical Approval Data

Replace with your actual data source (S3, database, etc.)

In [ ]:
# Option A: Load from S3
# import boto3
# s3 = boto3.client('s3')
# df = pd.read_csv('s3://your-bucket/approval_history.csv')

# Option B: Load from local file
# df = pd.read_csv('approval_history.csv')

# Option C: Generate synthetic data for demo
np.random.seed(42)
n_samples = 1000

# Synthetic approval request data
categories = ['software_license', 'travel_request', 'budget_variance', 
              'vendor_payment', 'pto_request', 'equipment_purchase']

df = pd.DataFrame({
    'request_id': range(n_samples),
    'request_text': [
        f"Request for {np.random.choice(categories)} - Amount: ${np.random.randint(100, 10000)}. "
        f"Justification: {'Business critical' if np.random.random() > 0.5 else 'Standard request'}."
        for _ in range(n_samples)
    ],
    'category': np.random.choice(categories, n_samples),
    'amount': np.random.randint(100, 10000, n_samples),
    'decision': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),  # 0=reject, 1=approve
    'approver_id': np.random.choice(['approver_1', 'approver_2', 'approver_3'], n_samples)
})

# Inject some label noise (realistic scenario)
noise_indices = np.random.choice(n_samples, size=int(n_samples * 0.1), replace=False)
df.loc[noise_indices, 'decision'] = 1 - df.loc[noise_indices, 'decision']

print(f"Loaded {len(df)} historical approval decisions")
print(f"\nDecision distribution:")
print(df['decision'].value_counts())
df.head()

## 3. Generate Embeddings

In [ ]:
# Load sentence transformer for text embeddings
print("Loading embedding model...")
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed request text
print("Generating embeddings...")
embeddings = encoder.encode(
    df['request_text'].tolist(),
    show_progress_bar=True
)

print(f"Embeddings shape: {embeddings.shape}")

## 4. Train Classifier and Get Cross-Validated Predictions

In [ ]:
# Train classifier
clf = LogisticRegression(max_iter=1000, n_jobs=-1)

# Get out-of-fold predicted probabilities (critical for Cleanlab)
print("Getting cross-validated predictions...")
pred_probs = cross_val_predict(
    clf,
    embeddings,
    df['decision'].values,
    cv=5,
    method='predict_proba',
    n_jobs=-1
)

print(f"Prediction shape: {pred_probs.shape}")
print(f"Sample predictions: {pred_probs[:5]}")

## 5. Cleanlab Analysis

In [ ]:
# Get label quality scores
labels = df['decision'].values

label_quality = get_label_quality_scores(labels, pred_probs)
df['label_quality'] = label_quality

print(f"Label quality statistics:")
print(f"  Mean: {label_quality.mean():.3f}")
print(f"  Std:  {label_quality.std():.3f}")
print(f"  Min:  {label_quality.min():.3f}")
print(f"  Max:  {label_quality.max():.3f}")

In [ ]:
# Find label issues
label_issues = find_label_issues(labels, pred_probs)
df['is_label_issue'] = label_issues

print(f"\nFound {label_issues.sum()} potential label issues ({100*label_issues.sum()/len(labels):.1f}%)")

In [ ]:
# Compute confident joint matrix
confident_joint = compute_confident_joint(labels, pred_probs)

print("Confident Joint Matrix (rows=given label, cols=estimated true label):")
print(pd.DataFrame(
    confident_joint,
    index=['Given: Reject', 'Given: Approve'],
    columns=['True: Reject', 'True: Approve']
))

In [ ]:
# Dataset health summary
health = health_summary(labels, pred_probs)

print("\nDataset Health Summary:")
print(f"  Overall health score: {health['overall_label_health_score']:.3f}")
print(f"  Number of label issues: {health['num_label_issues']}")

## 6. Examine Problematic Examples

In [ ]:
# Get worst quality examples
worst_examples = df.nsmallest(20, 'label_quality')[
    ['request_id', 'request_text', 'category', 'amount', 'decision', 
     'approver_id', 'label_quality', 'is_label_issue']
]

print("Top 20 Most Likely Mislabeled Decisions:")
print("="*80)
for idx, row in worst_examples.iterrows():
    decision_str = 'APPROVED' if row['decision'] == 1 else 'REJECTED'
    print(f"\n[Quality: {row['label_quality']:.3f}] Request #{row['request_id']}")
    print(f"  Category: {row['category']}, Amount: ${row['amount']}")
    print(f"  Decision: {decision_str} by {row['approver_id']}")
    print(f"  Text: {row['request_text'][:100]}...")

## 7. Approver Consistency Analysis

In [ ]:
# Analyze approver quality
approver_stats = df.groupby('approver_id').agg({
    'label_quality': ['mean', 'std', 'count'],
    'is_label_issue': 'sum'
}).round(3)

approver_stats.columns = ['avg_quality', 'std_quality', 'n_decisions', 'n_issues']
approver_stats['issue_rate'] = (approver_stats['n_issues'] / approver_stats['n_decisions']).round(3)

print("Approver Quality Analysis:")
print(approver_stats.sort_values('avg_quality'))

## 8. Export Cleaned Data

In [ ]:
# Create clean training set (exclude label issues)
df_clean = df[~df['is_label_issue']].copy()

print(f"Original dataset: {len(df)} examples")
print(f"Clean dataset: {len(df_clean)} examples")
print(f"Removed: {len(df) - len(df_clean)} noisy examples")

# Save for next notebook
df_clean.to_csv('approval_data_cleaned.csv', index=False)
np.save('embeddings_cleaned.npy', embeddings[~df['is_label_issue']])

print("\nCleaned data saved!")

## 9. Signals for YRSN Integration

These Cleanlab outputs will be used in the next notebook for YRSN decomposition:

In [ ]:
# Prepare YRSN input signals
cleanlab_signals = pd.DataFrame({
    'request_id': df['request_id'],
    'label_quality': df['label_quality'],
    'is_label_issue': df['is_label_issue'],
    'pred_prob_reject': pred_probs[:, 0],
    'pred_prob_approve': pred_probs[:, 1],
})

# Add derived signals
cleanlab_signals['normalized_margin'] = np.abs(pred_probs[:, 1] - pred_probs[:, 0])
cleanlab_signals['normalized_entropy'] = -np.sum(
    pred_probs * np.log(pred_probs + 1e-10), axis=1
) / np.log(2)  # Normalize by max entropy for binary

cleanlab_signals.to_csv('cleanlab_signals.csv', index=False)

print("Cleanlab signals for YRSN:")
cleanlab_signals.head(10)

## Summary

In this notebook we:
1. Loaded historical approval data
2. Generated text embeddings
3. Ran Cleanlab analysis to find label issues
4. Analyzed approver consistency
5. Exported clean training data
6. Prepared signals for YRSN decomposition

**Next**: Notebook 2 - YRSN Decomposition for Approval Context